This Notebook defines training of model using ignite method

In [1]:
!pip install pytorch-ignite
! pip install tensorboard

In [2]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from transformers import ViTFeatureExtractor, ViTForImageClassification,AutoFeatureExtractor,AdamW
from tqdm import tqdm
import torch
import numpy as np
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import gym
import numpy as np


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pylab as plt


%matplotlib inline

In [3]:
# Check if CUDA (GPU support) is available
if torch.cuda.is_available():

    print("CUDA is available. Using GPU.")
else:

    print("CUDA is not available. Using CPU.")


CUDA is available. Using GPU.


In [4]:
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k')

# Load pre-trained feature extractor
feature_extractor = AutoFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")

# Step 1: Modify classification head for the new number of classes
num_classes = 10  # Update with your new number of classes
model.classifier = torch.nn.Linear(model.config.hidden_size, num_classes)
model.config.image_size=32
# Move the model to the desired device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\sushant\anaconda3\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

In [5]:
index_to_label = {
    0: 'Airplane',
    1: 'Automobile',
    2: 'Bird',
    3: 'Cat',
    4: 'Deer',
    5: 'Dog',
    6: 'Frog',
    7: 'Horse',
    8: 'Ship',
    9: 'Truck'
}

In [6]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy,Loss

In [7]:
from ignite.handlers import ModelCheckpoint
from ignite.contrib.handlers import TensorboardLogger, global_step_from_engine

In [8]:
# Create the dataset and data loaders
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
model = model  # Initialize your ViT model here
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:

def score_function(engine):
    return (engine.state.metrics['accuracy'] + engine.state.metrics['auc']) / 2
    #return engine.state.metrics['accuracy']
val_metrics = {
    "accuracy": Accuracy(),
    "loss": Loss(criterion)
}

In [23]:


from ignite.engine import Engine


def train_step(engine, batch):
    model.train()
    inputs, targets = batch
    inputs, targets = inputs.to(device), targets.to(device)
    optimizer.zero_grad()
    outputs = model(inputs.to(device))
    logits = outputs.logits if hasattr(outputs, "logits") else outputs.last_hidden_state
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    loss = criterion(probabilities, targets)
    loss.backward()
    optimizer.step()
    return loss.item()
# Create Ignite trainer and evaluator
trainer = Engine(train_step)


In [24]:
def validation_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, y= batch
        y_pred = model(x.to(device))

    return y_pred, y


evaluator = Engine(validation_step)
# Attach metrics to the evaluators
for name, metric in val_metrics.items():
    metric.attach(evaluator, name)

for name, metric in val_metrics.items():
    metric.attach(evaluator, name)

In [13]:
# How many batches to wait before logging training status
log_interval = 100
@trainer.on(Events.ITERATION_COMPLETED(every=log_interval))
def log_training_loss(engine):
    print(f"Epoch[{engine.state.epoch}], Iter[{engine.state.iteration}] Loss: {engine.state.output:.2f}")


In [25]:
from ignite.engine import Events
from ignite.metrics import Accuracy

validate_every = 2


@trainer.on(Events.EPOCH_COMPLETED(every=validate_every))
def run_validation():
    evaluator.run(test_loader)

In [15]:
# def log_training_loss(engine):
#     print(f"Epoch[{engine.state.epoch}], Iter[{engine.state.iteration}] Loss: {engine.state.output:.2f}")

# trainer.add_event_handler(Events.ITERATION_COMPLETED, log_training_loss)

trainer vs. evaluator: In Ignite, the `trainer` refers to the engine responsible for the training loop, while the `evaluator` refers to the engine responsible for evaluation (validation or testing). They are separate engines with distinct roles.

The trainer is responsible for training the model using the training data loader and the specified training logic (train_step function).
The evaluator is responsible for evaluating the model using the validation or test data loader and the specified evaluation logic (validation_step function).

In [26]:
# Reinforcement Learning Setup
reward_window = []
best_accuracy = 0.0
@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    trainer.run(train_loader)
    metrics = trainer.state.metrics
    print(f"Training Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['loss']:.2f}")


@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(evaluator):
    evaluator.run(test_loader)
    metrics = evaluator.state.metrics
    print(f"Validation Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['loss']:.2f}")

@trainer.on(Events.EPOCH_COMPLETED)
def update_with_reward(evaluator):
    print("done")
    global best_accuracy
    
    # Use the validation accuracy as the reward
    reward = evaluator.state.metrics['accuracy']

    # Update the model based on the reward
    if reward > best_accuracy:
        best_accuracy = reward
        print(f"Updating model with reward: {reward}")
        torch.save(model.state_dict(), "best_model.pth")
# Accuracy().attach(evaluator, "accuracy")

In [17]:
!pip install jupyter-tensorboard


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for jupyter-tensorboard: filename=jupyter_tensorboard-0.2.0-py2.py3-none-any.whl size=15280 sha256=d65118cb5cf8bbbd1d2a8d3acdab08a5d2962d6ab098a654ff40f34eeaf6db9d
  Stored in directory: c:\users\sushant\appdata\local\pip\cache\wheels\9d\a7\53\b0a5a49c040337d6df9fb6bcdbea3d36eb823bd1c33c9e4ca2
Successfully built jupyter-tensorboard


Iginte Event Management

* Events.STARTED: Triggered when the engine is started.
* Events.COMPLETED: Triggered when the engine is completed.
* Events.EPOCH_STARTED: Triggered at the beginning of each epoch.
* Events.EPOCH_COMPLETED: Triggered at the end of each epoch.
* Events.ITERATION_STARTED: Triggered at the beginning of each iteration (batch).
* Events.ITERATION_COMPLETED: Triggered at the end of each iteration (batch).
* Events.EXCEPTION_RAISED: Triggered when an exception is raised in the engine.
* Events.TERMINATE: Triggered when the engine should terminate.
* Events.MODEL_CHECKPOINT: Triggered when a model checkpoint is about to be saved.
* Events.REDUCE_LR_ON_PLATEAU: Triggered during the learning rate reduction on plateau.

In [18]:
%load_ext tensorboard

In [27]:
model_checkpoint = ModelCheckpoint(
    "checkpoint",
    n_saved=2,
    filename_prefix="best",
    score_function=score_function,
    score_name="accuracy",
    global_step_transform=global_step_from_engine(trainer),
)

# Save the model after every epoch of val_evaluator is completed
evaluator.add_event_handler(Events.COMPLETED, model_checkpoint, {"model": model})

In [32]:
from ignite.contrib.handlers import TensorboardLogger, global_step_from_engine
tb_logger = TensorboardLogger(log_dir="tb-logger")

# Attach TensorBoard logger for loss, accuracy, and AUC
tb_logger.attach_output_handler(
    trainer,
    event_name=Events.ITERATION_COMPLETED(every=100),
    tag="training",
    output_transform=lambda loss: {"batch_loss": loss},
)

tb_logger.attach_output_handler(
    evaluator,
    event_name=Events.EPOCH_COMPLETED,
    tag="validation",
    metric_names=['loss', 'accuracy', 'auc'],
    global_step_transform=global_step_from_engine(trainer),
)


In [33]:
# Attach handler for plotting both evaluators' metrics after every epoch completes
for tag, evaluator in [("training", trainer), ("validation", evaluator)]:
    tb_logger.attach_output_handler(
        evaluator,
        event_name=Events.EPOCH_COMPLETED,
        tag=tag,
        metric_names="all",
        global_step_transform=global_step_from_engine(trainer),
    )

In [35]:
from ignite.contrib.handlers import ProgressBar

ProgressBar().attach(trainer)

In [37]:

# Run the training loop
trainer.run(train_loader, max_epochs=6)
tb_logger.close()

[1/6250]   0%|           [00:00<?]

In [ ]:
%tensorboard --logdir=./tb-logger